<a href="https://colab.research.google.com/github/lukmanaj/Mu-SHROOM/blob/main/Mu_SHROOM_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 9.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
!pip install -q evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.2 MB/s eta 0:00:00


In [22]:
# Install necessary libraries (uncomment if needed)
# !pip install transformers datasets

import torch
import json
import requests
import os
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from datasets import Dataset
import evaluate

# Define constants
LABEL_LIST = [0, 1]
MODEL_NAME = 'FacebookAI/xlm-roberta-base'

def download_and_load_jsonl(url):
    """Download a JSONL file from a raw GitHub URL and load it into a Dataset."""
    response = requests.get(url)
    response.raise_for_status()
    lines = response.text.strip().split('\n')
    data = [json.loads(line) for line in lines]
    return Dataset.from_list(data)

def tokenize_and_map_labels(examples, tokenizer):
    """Tokenize inputs and map labels to token-level annotations."""
    tokenized_inputs = tokenizer(examples['model_output_text'], return_offsets_mapping=True, padding=True, truncation=True)
    offset_mappings = tokenized_inputs['offset_mapping']
    all_labels = examples['hard_labels']
    tok_labels_batch = []
    for batch_idx in range(len(offset_mappings)):
        offset_mapping = offset_mappings[batch_idx]
        hard_labels = all_labels[batch_idx]
        tok_labels = [0] * len(offset_mapping)
        for idx, start_end in enumerate(offset_mapping):
            start = start_end[0]
            end = start_end[1]
            for (label_start, label_end) in hard_labels:
                if start >= label_start and end <= label_end:
                    tok_labels[idx] = 1
        tok_labels_batch.append(tok_labels)
    tokenized_inputs['labels'] = tok_labels_batch
    return tokenized_inputs

def train_model(test_lang, train_urls, val_url, output_dir):
    """Train the model using datasets downloaded from GitHub."""
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForTokenClassification.from_pretrained(MODEL_NAME, num_labels=2)

    # Load datasets
    train_dataset = download_and_load_jsonl(train_url)
    val_dataset = download_and_load_jsonl(val_url)

    # Tokenize datasets
    train_dataset = train_dataset.map(lambda x: tokenize_and_map_labels(x, tokenizer), batched=True)
    val_dataset = val_dataset.map(lambda x: tokenize_and_map_labels(x, tokenizer), batched=True)

    # Define training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        evaluation_strategy='epoch',
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=5,
        weight_decay=0.01,
    )

    # Define the metric
    metric = metric = evaluate.load('seqeval') # Load the evaluation metric


    # def compute_metrics(p):
    #     predictions, labels = p
    #     predictions = torch.argmax(torch.tensor(predictions), dim=2)
    #     true_labels = [[LABEL_LIST[l] for l in label if l != -100] for label in labels]
    #     true_predictions = [
    #         [LABEL_LIST[p] for (p, l) in zip(prediction, label) if l != -100]
    #         for prediction, label in zip(predictions, labels)
    #     ]
    #     print("True Predictions:", true_predictions[:2])  # Print the first 2 predictions
    #     print("True Labels:", true_labels[:2])  # Print the first 2 labels


    #     results = metric.compute(predictions=true_predictions, references=true_labels)
    #     return {
    #         "precision": results["overall_precision"],
    #         "recall": results["overall_recall"],
    #         "f1": results["overall_f1"],
    #         "accuracy": results["overall_accuracy"],
    #     }

    def compute_metrics(p):
        predictions, labels = p

        # Create a mapping for numeric to string labels
        label_map = {0: 'O', 1: 'B-LABEL'}  # Modify the labels as per your dataset
        predictions = torch.argmax(torch.tensor(predictions), dim=2).tolist()

        # Convert predictions and labels to string-based labels, filtering out special tokens (-100)
        true_predictions = [
            [label_map[p] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]
        true_labels = [
            [label_map[l] for l in label if l != -100]
            for label in labels
        ]

        # Use the `seqeval` metric
        results = metric.compute(predictions=true_predictions, references=true_labels)
        return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }


    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )

    # Train and evaluate
    trainer.train()
    trainer.evaluate()
    print(f"Model trained and evaluated successfully. Checkpoint saved in {output_dir}")

def test_model(test_lang, model_path, test_url):
    """Test the model using a test dataset from GitHub."""
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    model = AutoModelForTokenClassification.from_pretrained(model_path)

    # Load test dataset
    test_dataset = download_and_load_jsonl(test_url)

    # Tokenize test dataset
    inputs = tokenizer(test_dataset['model_output_text'], padding=True, truncation=True, return_offsets_mapping=True, return_tensors="pt")

    # Get predictions
    model.eval()
    with torch.no_grad():
        outputs = model(inputs.input_ids)
    preds = torch.argmax(outputs.logits, dim=2)
    probs = F.softmax(outputs.logits, dim=2)

    # Map predictions to character spans
    hard_labels_all = {}
    soft_labels_all = {}
    predictions_all = []
    for i, pred in enumerate(preds):
        hard_labels_sample = []
        soft_labels_sample = []
        positive_indices = torch.nonzero(pred == 1, as_tuple=False)
        offset_mapping = inputs['offset_mapping'][i]
        for j, offset in enumerate(offset_mapping):
            soft_labels_sample.append({'start': offset[0].item(), 'end': offset[1].item(), 'prob': probs[i][j][1].item()})
            if j in positive_indices:
                hard_labels_sample.append((offset[0].item(), offset[1].item()))
        soft_labels_all[test_dataset['id'][i]] = soft_labels_sample
        hard_labels_all[test_dataset['id'][i]] = hard_labels_sample
        predictions_all.append({'id': test_dataset['id'][i], 'hard_labels': hard_labels_sample, 'soft_labels': soft_labels_sample})

    # Save results
    with open(f"{test_lang}-hard_labels.json", 'w') as f:
        json.dump(hard_labels_all, f)
    with open(f"{test_lang}-soft_labels.json", 'w') as f:
        json.dump(soft_labels_all, f)
    with open(f"{test_lang}-pred.jsonl", 'w') as f:
        for pred_dict in predictions_all:
            print(json.dumps(pred_dict), file=f)
    print(f"Labels saved to {test_lang}-hard_labels.json and {test_lang}-soft_labels.json")
    print(f"Prediction file saved to {test_lang}-pred.jsonl")


In [23]:
# Example GitHub URLs
train_url = "https://raw.githubusercontent.com/lukmanaj/Mu-SHROOM/refs/heads/main/val/mushroom.en-val.v2.jsonl"  # "https://raw.githubusercontent.com/lukmanaj/Mu-SHROOM/refs/heads/main/val/mushroom.de-val.v2.jsonl",


val_url = "https://raw.githubusercontent.com/lukmanaj/Mu-SHROOM/refs/heads/main/val/mushroom.ar-val.v2.jsonl"
test_url = "https://raw.githubusercontent.com/lukmanaj/Mu-SHROOM/refs/heads/main/val/mushroom.ar-val.v2.jsonl"


In [24]:
output_dir = "/content/results"
train_model(test_lang='ar', train_urls=train_url, val_url=val_url, output_dir=output_dir)


Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at FacebookAI/xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.512615,0.000000,0.000000,0.000000,0.886992
2,No log,0.424998,0.000000,0.000000,0.000000,0.886992
3,No log,0.338169,0.000000,0.000000,0.000000,0.886992
4,No log,0.280877,0.000000,0.000000,0.000000,0.886992
5,No log,0.279328,0.000000,0.000000,0.000000,0.886992


/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Model trained and evaluated successfully. Checkpoint saved in /content/results


In [26]:
fine_tuned_model = "/content/results/checkpoint-20"
test_model(test_lang='ar', model_path=fine_tuned_model, test_url=test_url)

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


Labels saved to ar-hard_labels.json and ar-soft_labels.json
Prediction file saved to ar-pred.jsonl
